In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import time

In [2]:
previsores = pd.read_csv('entradas_breast.csv')
previsores.head(3)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758


In [3]:
previsores.shape

(569, 30)

In [4]:
previsores.drop(previsores.tail(9).index,inplace=True) # drop last n rows
previsores.shape

(560, 30)

In [5]:
classe = pd.read_csv('saidas_breast.csv')
classe.head(3)

,0
0,0
1,0
2,0


In [6]:
classe.shape

(569, 1)

In [7]:
classe.drop(classe.tail(9).index,inplace=True) # drop last n rows
classe.shape

(560, 1)

In [8]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

In [9]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [10]:
def criarRede(drop1, drop2, clipvalue):
    # cleanup
    k.clear_session()
    
    classificador = Sequential()
    
    classificador.add(Dense(units = 32, activation = 'relu', 
                        kernel_initializer = 'normal', input_dim = 30))
    
    classificador.add(Dropout(drop1))
    
    classificador.add(Dense(units = 32, activation = 'relu', 
                        kernel_initializer = 'normal'))
    
    classificador.add(Dropout(drop2))
    
    classificador.add(Dense(units = 1, activation = 'sigmoid'))
    
    otimizador = keras.optimizers.Adam(lr = 0.001, decay = 0.0001, clipvalue = clipvalue)
    
    classificador.compile(optimizer = otimizador, 
                          loss = 'binary_crossentropy',
                          metrics = ['binary_accuracy'])
                          #run_eagerly=True)
    
    return classificador

In [11]:
classificador = KerasClassifier(build_fn = criarRede)
parametros = {'batch_size': [10],
              'epochs': [100],
              'callbacks': [ClearMemory()],
              'clipvalue': [0.5, 0.1, 0.05],
              'drop1': [0.2, 0.25, 0.28],
              'drop2': [0.2, 0.25, 0.28]}

In [12]:
from sklearn.utils import parallel_backend

In [13]:
grid_search = GridSearchCV(estimator = classificador,
                           param_grid = parametros,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs=1)

In [ ]:
start = time.time()
with parallel_backend('threading'):
    grid_search = grid_search.fit(previsores, classe)

Epoch 1/100
45/45 [==============================] - 0s 4ms/step - loss: 5.2759 - binary_accuracy: 0.5714
Epoch 2/100
45/45 [==============================] - 0s 3ms/step - loss: 2.2398 - binary_accuracy: 0.6339
Epoch 3/100
45/45 [==============================] - 0s 3ms/step - loss: 1.3618 - binary_accuracy: 0.6384
Epoch 4/100
45/45 [==============================] - 0s 3ms/step - loss: 1.1342 - binary_accuracy: 0.6384
Epoch 5/100
45/45 [==============================] - 0s 4ms/step - loss: 0.8961 - binary_accuracy: 0.6674
Epoch 6/100
45/45 [==============================] - 0s 3ms/step - loss: 1.0076 - binary_accuracy: 0.6719
Epoch 7/100
45/45 [==============================] - 0s 4ms/step - loss: 0.8489 - binary_accuracy: 0.7299
Epoch 8/100
45/45 [==============================] - 0s 3ms/step - loss: 0.8968 - binary_accuracy: 0.6920
Epoch 9/100
45/45 [==============================] - 0s 3ms/step - loss: 0.9719 - binary_accuracy: 0.6964
Epoch 10/100
45/45 [==========================

In [ ]:
end = time.time()
total = (end - start)/60
print('{}min'.format(total))

In [ ]:
total = (end - start)/60
print('{:.2f}min'.format(total))

In [15]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'batch_size': 10, 'clipvalue': 0.5, 'drop1': 0.2, 'drop2': 0.2, 'epochs': 100}

In [16]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.875

In [35]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'batch_size': 10,
 'clipvalue': 0.5,
 'drop1': 0.2,
 'drop2': 0.2,
 'epochs': 100,
 'neurons1': 8,
 'neurons2': 32}

In [36]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.8858562335041142

In [18]:
melhores_parametros = grid_search.best_params_
melhores_parametros

{'batch_size': 10,
 'clipvalue': 0.5,
 'decay': 0.0001,
 'drop': 0.2,
 'epochs': 100,
 'lr': 0.001,
 'neurons': 32}

In [19]:
melhor_precisao = grid_search.best_score_
melhor_precisao

0.8787300108678776